In [1]:
import numpy as np
import pandas as pd

In [2]:
from src.data import all_functions as af

[2021-03-08 19:33:25,021 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-08 19:33:25,037 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 18.143616914749146
[2021-03-08 19:33:25,037 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [3]:
import requests
import pandas as pd
from src.data import all_functions as af

PLAYER_COLUMN = "PLAYER"
TEAM_KEY = "team_key"
PLAYER_ID_KEY = "player_id"
NAME_KEY = "name"
MEAN_ROW = "mean"
NAME_KEY = "name"
LSCD_KEY = "lscd"
MSCD_KEY = "mscd"
G_KEY = "g"
GID_KEY = "gid"
GAME_ID_KEY = "game_id"
GDATE_KEY = "gdte"
GAME_DATE_KEY= "game_date"
V_KEY = "v"
TID_KEY = "tid"
VISITORS_TEAM_ID_KEY = "visitors_tid"
VISITORS_KEY =  "visitors"
TC_KEY = "tc"
TN_KEY = "tn"
H_KEY = "h"
HOME_TEAM_ID_KEY = "home_tid"
HOME_KEY = "home"
TOTAL_ROW = "total"
INDEX_COLUMN = "index"
NBA_NAME_LA_CLIPPERS = "LA Clippers"
YAHOO_NAME_LA_CLIPPERS = "Los Angeles Clippers"
COLUMN_9CAT_DECIMAL_FORMAT = {af.FIELD_GOAL_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.FREE_THROW_PERCENTAGE_COLUMN : af.THREE_DECIMAL_FORMAT,
     af.THREES_MADE_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.POINTS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.REBOUNDS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.ASSITS_COLUMN : af.TWO_DECIMAL_FORMAT, af.STEALS_COLUMN : af.TWO_DECIMAL_FORMAT,
     af.BLOCKS_COLUMN : af.TWO_DECIMAL_FORMAT, af.TURNOVERS_COLUMN : af.TWO_DECIMAL_FORMAT}

def get_game_information_in_month(nba_json):
    """
    Function loops through a month (n) in nba_json and creates a dataframe containing game
    information, game_id, home/visitor team/team_id and game_date for each month.
    param nba_json (dictionary): dictionary containing game information
    returns game_dataframe (pandas dataframe): dataframe containing game information for a month
    """
    game_dataframe = pd.DataFrame()
    for n in range(len(nba_json)):
        game_dictionary = dict()
        game_dictionary[GID_KEY] = nba_json[n][GID_KEY]
        game_dictionary[GDATE_KEY] = nba_json[n][GDATE_KEY]
        game_dictionary[VISITORS_TEAM_ID_KEY] = nba_json[n][V_KEY][TID_KEY]
        game_dictionary[VISITORS_KEY] = nba_json[n][V_KEY][TC_KEY] + " " \
                                        + nba_json[n][V_KEY][TN_KEY]
        game_dictionary[HOME_TEAM_ID_KEY] = nba_json[n][H_KEY][TID_KEY]
        game_dictionary[HOME_KEY] = nba_json[n][H_KEY][TC_KEY] + " " \
                                    + nba_json[n][H_KEY][TN_KEY]
        game_dataframe = game_dataframe.append(game_dictionary, ignore_index=True)
    return game_dataframe

def get_game_information_in_season(season):
    """
    Function calls the NBA endpoint that returns the season schedule. It then loops through each
    month in that schedule to create a dataframe containing game information, game_id,
    home/visitor team/team_id and game_date for the season.
    param season (string): season for schedule
    returns season_games_dataframe (pandas dataframe): dataframe containing game information for
    a season
    """
    r = requests.get(f"http://data.nba"
                     f".com/data/10s/v2015/json/mobile_teams/nba/{season}/league/00_full_schedule"
                     f".json")
    game_information_json = r.json()[LSCD_KEY]
    season_games_dataframe = pd.DataFrame()
    for n in range(len(game_information_json)):
        month_n_game_dataframe = get_game_information_in_month\
            (game_information_json[n][MSCD_KEY][G_KEY])
        season_games_dataframe = season_games_dataframe.append(month_n_game_dataframe)
    return season_games_dataframe

def get_week_current_week_information():
    """
    Function finds the current Yahoo Fantasy week number and its start and end date.
    return current_fantasy_week (int): Yahoo Fantasy week number
    return week_start_date (string): start date of Yahoo Fantasy week number
    return week_end_date (string): end date of Yahoo Fantasy week number
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    current_fantasy_week = league.current_week()
    week_start_date = league.week_date_range(current_fantasy_week)[0]
    week_end_date = league.week_date_range(current_fantasy_week)[1]
    return current_fantasy_week, week_start_date, week_end_date

def get_next_week_information():
    """
    Function finds the next Yahoo Fantasy week number and its start and end date.
    return next_fantasy_week (int): Yahoo Fantasy week number
    return week_start_date (string): start date of Yahoo Fantasy week number
    return week_end_date (string): end date of Yahoo Fantasy week number
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    next_fantasy_week = league.current_week() + 1
    week_start_date = league.week_date_range(next_fantasy_week)[0]
    week_end_date = league.week_date_range(next_fantasy_week)[1]
    return next_fantasy_week, week_start_date, week_end_date

def get_player_ids_names_in_fantasy_team(team_dict):
    """
    Function finds the Yahoo Fantasy player ids and player names rostered by a team in the league
    param team_dict (dictionary): dictionary containing information about a owners team
    return player_id_name_team_list (list): Yahoo Fantasy player ids and names
    return player_name_list (list): Yahoo Fantasy player names
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team = league.to_team(team_dict[TEAM_KEY])
    team_roster = team.roster()
    player_id_name_team_list = [{PLAYER_ID_KEY : player[PLAYER_ID_KEY], NAME_KEY : player[NAME_KEY]}
                           for player in team_roster]
    player_name_list = [player[NAME_KEY] for player in player_id_name_team_list]
    return player_id_name_team_list, player_name_list

def get_fantasy_week_games_dataframe(season_schedule_dataframe, week_start_date, week_end_date):
    """
    Function filters the entire NBA season schedule on a start/end date to find the scheduled
    games between specific dates. It returns the amount of games each NBA team plays during that
    period of time.
    param season_schedule_dataframe (pandas dataframe): dataframe containing the entire seasons
    NBA schedule
    param week_start_date (string): date to begin filtering by
    param week_start_date (string): date to end filtering by
    return current_fantasy_games_week (pandas dataframe) dataframe containing the scheduled games
    in specified param dates
    return team_game_counts (pandas series): series containing count of games for each NBA team
    """
    filtered_games_week = season_schedule_dataframe[(season_schedule_dataframe[GDATE_KEY] >=
                                                   str(week_start_date)) &
                                                  (season_schedule_dataframe[GDATE_KEY] <=
                                                   str(week_end_date))]
    team_game_counts = filtered_games_week[HOME_KEY].append\
        (filtered_games_week[VISITORS_KEY]).value_counts()
    return filtered_games_week, team_game_counts

def get_player_games(team_game_counts_series, player_name_list):
    """
    Function creates a dictionary containing key/value pairs of player names/games in a series of
    NBA team game counts
    param team_game_counts_series (pandas series): series of team name and games playing
    param player_name_list (list): list of player names
    return player_games_dictionary (dictionary): key/value pairing of player names/games playing
    """
    player_games_dictionary = dict()
    team_game_counts_dataframe = pd.DataFrame(team_game_counts_series)
    team_game_counts_dataframe.reset_index(inplace=True)
    team_game_counts_dataframe = team_game_counts_dataframe.replace(NBA_NAME_LA_CLIPPERS,
                                                                    YAHOO_NAME_LA_CLIPPERS)
    for player in player_name_list:
        team = af.yahoo_player_team_and_jersey(player)[0]
        try:
            player_games_dictionary[player] = \
                team_game_counts_dataframe[team_game_counts_dataframe["index"] == team][0].iloc[0]
        except IndexError:
            player_games_dictionary[player] = 0
    return player_games_dictionary

def get_team_player_game_dataframe(team_dict, week="next", season=2020):
    """

    @param team_dict:
    @param week:
    @param season:
    @return:
    """
    if week == "next":
        fantasy_week, week_start_date, week_end_date = get_next_week_information()
    elif week == "current":
        fantasy_week, week_start_date, week_end_date = get_week_current_week_information()
    else:
        raise ValueError("Parameter 'week' must be 'current' or 'next'.")
    season_games_dataframe = get_game_information_in_season(season) # Run once only
    filtered_games_week, team_game_counts = get_fantasy_week_games_dataframe\
        (season_games_dataframe, week_start_date, week_end_date)
    player_id_name_team_list, player_name_list = get_player_ids_names_in_fantasy_team(team_dict)
    player_games_dictionary = get_player_games(team_game_counts, player_name_list)
    return filtered_games_week, player_games_dictionary

In [ ]:
filtered_games_week, player_games_dictionary = get_team_player_game_dataframe(af.NUNN_OF_YALL_BETTA,
                                                                            week="current",
                                                                              season=2020)

In [105]:
player_games

,0
Malcolm Brogdon,4
Duncan Robinson,4
Patty Mills,4
Gary Trent Jr.,4
Jae Crowder,4
Jeremy Lamb,4
Enes Kanter,4
Nikola Jokic,4
Cameron Johnson,4
Jimmy Butler,4


In [108]:
player_games = pd.DataFrame.from_dict(player_games_dictionary, orient="index")
df = player_games.loc[player_games.index.repeat(player_games[0])]
df
# df.merge(nunn, left_on="index", right_on="PLAYER")

,0
Malcolm Brogdon,4
Malcolm Brogdon,4
Malcolm Brogdon,4
Malcolm Brogdon,4
Duncan Robinson,4
Duncan Robinson,4
Duncan Robinson,4
Duncan Robinson,4
Patty Mills,4
Patty Mills,4


In [ ]:
sc = af.yahoo_fantasy_api_authentication()
league = af.yahoo_fantasy_league(sc)

In [80]:
nunn = af.team_9cat_average_stats(af.NUNN_OF_YALL_BETTA, league)

In [83]:
nunn

,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
0,Malcolm Brogdon,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09
1,Duncan Robinson,0.429,0.840,3.25,12.81,3.94,1.61,0.47,0.22,1.33
2,Patty Mills,0.448,0.900,2.75,13.31,1.81,2.66,0.62,0.03,1.06
3,Gary Trent Jr.,0.423,0.778,3.06,15.21,2.15,1.48,0.76,0.15,0.73
4,Jae Crowder,0.402,0.829,2.41,10.12,5.00,2.12,0.84,0.25,0.91
5,Jeremy Lamb,0.473,0.941,1.62,11.33,3.71,1.62,0.86,0.52,0.76
6,Enes Kanter,0.594,0.750,0.00,11.91,11.86,1.14,0.51,0.74,1.17
7,Nikola Jokic,0.567,0.881,1.56,27.14,11.03,8.58,1.61,0.64,3.17
8,Cameron Johnson,0.437,0.865,2.15,10.44,3.38,1.35,0.65,0.29,0.76
9,Jimmy Butler,0.454,0.863,0.36,20.55,7.55,7.82,1.91,0.36,2.32


In [40]:
nunn.drop("PLAYER", axis=1, inplace=True)
nunn.drop("mean", inplace=True)
nunn = nunn.apply(pd.to_numeric)

,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
0,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09
1,0.429,0.840,3.25,12.81,3.94,1.61,0.47,0.22,1.33
2,0.448,0.900,2.75,13.31,1.81,2.66,0.62,0.03,1.06
3,0.423,0.778,3.06,15.21,2.15,1.48,0.76,0.15,0.73
4,0.402,0.829,2.41,10.12,5.00,2.12,0.84,0.25,0.91
5,0.473,0.941,1.62,11.33,3.71,1.62,0.86,0.52,0.76
6,0.594,0.750,0.00,11.91,11.86,1.14,0.51,0.74,1.17
7,0.567,0.881,1.56,27.14,11.03,8.58,1.61,0.64,3.17
8,0.437,0.865,2.15,10.44,3.38,1.35,0.65,0.29,0.76
9,0.454,0.863,0.36,20.55,7.55,7.82,1.91,0.36,2.32


In [118]:
sc = af.yahoo_fantasy_api_authentication()
league = af.yahoo_fantasy_league(sc)
team_9cat_averages = af.team_9cat_average_stats(af.NUNN_OF_YALL_BETTA, league)
team_9cat_averages.drop("mean", inplace=True)
filtered_games_week, player_games_dictionary = get_team_player_game_dataframe(af.NUNN_OF_YALL_BETTA,
                                                                              week="current",
                                                                              season=2020)
player_games = pd.DataFrame.from_dict(player_games_dictionary, orient="index")
player_games = player_games.loc[player_games.index.repeat(player_games[0])]
player_games.reset_index(inplace=True)
player_games.columns = [["PLAYER", "G"]]
# simulation_dataset = player_games.merge(team_9cat_averages, on="PLAYER")

[2021-03-08 21:00:48,006 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-08 21:00:48,011 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1055.4016585350037
[2021-03-08 21:00:48,013 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-08 21:01:46,320 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-08 21:01:46,324 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1113.7149488925934
[2021-03-08 21:01:46,325 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-08 21:01:52,384 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-08 21:01:52,388 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1119.7789208889008
[2021-03-08 21:01:52,388 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-08 21:01:53,330 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-08 21:01:53,333 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1120.724420785904
[2021-03-08 21:01:53,334 DEBUG] [yahoo_oauth.oaut

ValueError: The column label 'PLAYER' is not unique.
For a multi-index, the label must be a tuple with elements corresponding to each level.

In [124]:
# player_games.merge(team_9cat_averages, left_on="PLAYER", right_on="PLAYER")
team_9cat_averages

,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
0,Malcolm Brogdon,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09
1,Duncan Robinson,0.429,0.840,3.25,12.81,3.94,1.61,0.47,0.22,1.33
2,Patty Mills,0.448,0.900,2.75,13.31,1.81,2.66,0.62,0.03,1.06
3,Gary Trent Jr.,0.423,0.778,3.06,15.21,2.15,1.48,0.76,0.15,0.73
4,Jae Crowder,0.402,0.829,2.41,10.12,5.00,2.12,0.84,0.25,0.91
5,Jeremy Lamb,0.473,0.941,1.62,11.33,3.71,1.62,0.86,0.52,0.76
6,Enes Kanter,0.594,0.750,0.00,11.91,11.86,1.14,0.51,0.74,1.17
7,Nikola Jokic,0.567,0.881,1.56,27.14,11.03,8.58,1.61,0.64,3.17
8,Cameron Johnson,0.437,0.865,2.15,10.44,3.38,1.35,0.65,0.29,0.76
9,Jimmy Butler,0.454,0.863,0.36,20.55,7.55,7.82,1.91,0.36,2.32


In [127]:
team_9cat_averages.merge(player_games, left_on="PLAYER", right_on="PLAYER")

ValueError: The column label 'PLAYER' is not unique.
For a multi-index, the label must be a tuple with elements corresponding to each level.

In [129]:
team_9cat_averages.to_clipboard(index=False)
team_9cat_averages=pd.read_clipboard(sep='\t')

player_games.to_clipboard(index=False)
player_games=pd.read_clipboard(sep='\t')

In [130]:
pd.merge(team_9cat_averages, player_games, on=["PLAYER"])

,PLAYER,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV,G
0,Malcolm Brogdon,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09,5
1,Malcolm Brogdon,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09,5
2,Malcolm Brogdon,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09,5
3,Malcolm Brogdon,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09,5
4,Malcolm Brogdon,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09,5
...,...,...,...,...,...,...,...,...,...,...,...
67,Pascal Siakam,0.453,0.819,1.20,20.07,7.47,4.77,1.17,0.70,2.17,5
68,Pascal Siakam,0.453,0.819,1.20,20.07,7.47,4.77,1.17,0.70,2.17,5
69,Pascal Siakam,0.453,0.819,1.20,20.07,7.47,4.77,1.17,0.70,2.17,5
70,Pascal Siakam,0.453,0.819,1.20,20.07,7.47,4.77,1.17,0.70,2.17,5


In [4]:
def make_simulation_dataset(team_dict, week="current"):
    """

    @param team_dict:
    @param week:
    @return:
    """
    sc = af.yahoo_fantasy_api_authentication()
    league = af.yahoo_fantasy_league(sc)
    team_9cat_averages = af.team_9cat_average_stats(team_dict, league)
    team_9cat_averages.drop("mean", inplace=True)
    filtered_games_week, player_games_dictionary = get_team_player_game_dataframe(team_dict,
                                                                                  week=week,
                                                                                  season=2020)
    player_games = pd.DataFrame.from_dict(player_games_dictionary, orient="index")
    player_games = player_games.loc[player_games.index.repeat(player_games[0])]
    player_games.reset_index(inplace=True)
    player_games.columns = [["PLAYER", "G"]]
    team_9cat_averages.to_clipboard(index=False)
    team_9cat_averages=pd.read_clipboard(sep='\t')
    player_games.to_clipboard(index=False)
    player_games=pd.read_clipboard(sep='\t')
    simulation_dataset = pd.merge(team_9cat_averages, player_games, on=["PLAYER"])
    simulation_dataset.drop(["PLAYER", "G"], axis=1, inplace=True)
    simulation_dataset = simulation_dataset.apply(pd.to_numeric)
    return simulation_dataset

In [132]:
simulation_dataset = make_simulation_dataset(af.NUNN_OF_YALL_BETTA, week="current")
simulation_dataset

[2021-03-08 21:09:18,815 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-08 21:09:18,817 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1566.2080564498901
[2021-03-08 21:09:18,818 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-08 21:10:16,873 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-08 21:10:16,877 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1624.2677590847015
[2021-03-08 21:10:16,878 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-08 21:10:25,193 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-08 21:10:25,197 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1632.5879747867584
[2021-03-08 21:10:25,197 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-08 21:10:28,527 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-08 21:10:28,529 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 1635.9200959205627
[2021-03-08 21:10:28,530 DEBUG] [yahoo_oauth.oau

,FG_PCT,FT_PCT,FG3M,PTS,REB,AST,STL,BLK,TOV
0,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09
1,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09
2,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09
3,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09
4,0.447,0.897,2.41,21.21,4.65,6.24,1.18,0.24,2.09
...,...,...,...,...,...,...,...,...,...
67,0.453,0.819,1.20,20.07,7.47,4.77,1.17,0.70,2.17
68,0.453,0.819,1.20,20.07,7.47,4.77,1.17,0.70,2.17
69,0.453,0.819,1.20,20.07,7.47,4.77,1.17,0.70,2.17
70,0.453,0.819,1.20,20.07,7.47,4.77,1.17,0.70,2.17


In [5]:
gaussian_stats = ['FG_PCT', 'FT_PCT']
poison_stats = ['FG3M', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'TOV']

def simulate_stats(team_df, n_reps):

    pois_stats = team_df.loc[:, poison_stats].values
    gaus_stats = team_df.loc[:, gaussian_stats].values
    pois = np.random.poisson(lam=pois_stats,
                             size=(n_reps, *pois_stats.shape))
    pois = np.maximum(0, pois)
    gaus = np.random.normal(loc=gaus_stats, scale=0.2 * gaus_stats,
                            size=(n_reps, *gaus_stats.shape))
    gaus = np.clip(gaus, 0, 1)

    aggr_pois_stats = pois.sum(axis=1)
    aggr_gaus_stats = gaus.mean(axis=1)
    aggr_stats = np.concatenate((aggr_gaus_stats, aggr_pois_stats),
                                axis=1)
    return aggr_stats

In [133]:
nunn_simulated = simulate_stats(simulation_dataset, 10000)
nunn_simulated

array([[  0.46597222,   0.82735082, 141.        , ...,  71.        ,
         29.        ,  94.        ],
       [  0.45297182,   0.81913327, 116.        , ...,  67.        ,
         22.        , 113.        ],
       [  0.46966351,   0.78848219, 152.        , ...,  66.        ,
         30.        , 142.        ],
       ...,
       [  0.46886232,   0.79490553, 106.        , ...,  61.        ,
         38.        , 129.        ],
       [  0.47487524,   0.80112521, 131.        , ...,  71.        ,
         37.        , 101.        ],
       [  0.47467781,   0.80851758, 153.        , ...,  71.        ,
         30.        , 118.        ]])

In [63]:
crabbe = af.team_9cat_average_stats(af.CRABBEHERBYTHEPUSSY, league)
crabbe.drop("PLAYER", axis=1, inplace=True)
crabbe.drop("mean", inplace=True)
crabbe = crabbe.apply(pd.to_numeric)

In [65]:
crabbe_simulated = simulate_stats(crabbe, 100000)

In [68]:
perc_win = ((nunn_simulated > crabbe_simulated).sum(axis=0)
            / 100000 * 100)
perc_win

array([26.314, 45.202, 71.874, 68.78 , 28.811, 87.807, 60.133,  4.706,
       42.451])

In [71]:
perc_win_df = pd.DataFrame(perc_win,
                           index=gaussian_stats + poison_stats,
                           columns=['WinProb'])
perc_win_df.loc['TOV'] = 100 - perc_win_df.loc['TOV']
perc_win_df


,WinProb
FG_PCT,26.314
FT_PCT,45.202
FG3M,71.874
PTS,68.780
REB,28.811
AST,87.807
STL,60.133
BLK,4.706
TOV,57.549


In [6]:
gaussian_stats = ['FG_PCT', 'FT_PCT']
poison_stats = ['FG3M', 'PTS', 'REB', 'AST', 'STL', 'BLK', 'TOV']

def matchup_simulation(team1_dict, team2_dict, n_reps=100000, week="current"):
    """

    @param team1_dict:
    @param team2_dict:
    @param n_reps:
    @return:
    """
    team1_simulation_dataset = make_simulation_dataset(team1_dict, week=week)
    team2_simulation_dataset = make_simulation_dataset(team2_dict, week=week)
    team1_sim_stats = simulate_stats(team1_simulation_dataset, n_reps)
    team2_sim_stats = simulate_stats(team2_simulation_dataset, n_reps)
    perc_win = ((team1_sim_stats > team2_sim_stats).sum(axis=0)
                / n_reps * 100)
    perc_win_df = pd.DataFrame(perc_win,
                               index=gaussian_stats + poison_stats,
                               columns=['WinProb'])
    perc_win_df.loc['TOV'] = 100 - perc_win_df.loc['TOV']
    return perc_win_df

In [7]:
simulated_cats = matchup_simulation(af.NUNN_OF_YALL_BETTA, af.RUSTY_CUNTBROOKS, n_reps=100000,
                                    week="current")

[2021-03-08 22:42:54,745 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-08 22:42:54,751 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3232.9738824367523
[2021-03-08 22:42:54,752 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-08 22:43:51,860 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-08 22:43:51,865 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3290.0883646011353
[2021-03-08 22:43:51,866 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-08 22:43:59,665 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-08 22:43:59,668 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3297.8913543224335
[2021-03-08 22:43:59,669 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID
[2021-03-08 22:44:00,675 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2021-03-08 22:44:00,678 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 3298.900946855545
[2021-03-08 22:44:00,678 DEBUG] [yahoo_oauth.oaut

In [8]:
simulated_cats



,WinProb
FG_PCT,5.583
FT_PCT,98.828
FG3M,98.582
PTS,72.192
REB,36.843
AST,86.626
STL,27.897
BLK,1.659
TOV,72.551
